In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)


In [3]:
!pip install lightgbm

In [3]:
from lightgbm import LGBMRegressor

In [5]:
#!conda install -c conda-forge lightgbm

In [4]:
lgbm = LGBMRegressor()
lgbm_model = lgbm.fit(X_train, y_train)

# Tahmin

In [5]:
y_pred = lgbm_model.predict(X_test, 
                            num_iteration = lgbm_model.best_iteration_)

In [8]:
np.sqrt(mean_squared_error(y_test, y_pred))

363.8712087611089

# Model Tuning

In [7]:
lgbm_model

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [8]:
lgbm_grid = {
    'colsample_bytree': [0.4, 0.5,0.6,0.9,1],
    'learning_rate': [0.01, 0.1, 0.5,1],
    'n_estimators': [20, 40, 100, 200, 500,1000],
    'max_depth': [1,2,3,4,5,6,7,8] }

lgbm = LGBMRegressor()
lgbm_cv_model = GridSearchCV(lgbm, lgbm_grid, cv=10, n_jobs = -1, verbose = 2)

In [9]:
lgbm_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 960 candidates, totalling 9600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 448 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 1014 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2634 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 3688 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 4887 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 6265 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 7803 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 9505 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 9593 out of 9600 | elapsed:  7.7min remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 9600 out of 9600 | elapsed:  7.7min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6, 0.9, 1],
                         'learning_rate': [0.01, 0.1, 0.5, 1],
                         'max_depth': 

In [10]:
lgbm_cv_model.best_params_

{'colsample_bytree': 0.6,
 'learning_rate': 0.1,
 'max_depth': 7,
 'n_estimators': 40}

In [11]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.1, 
                           max_depth = 7, 
                           n_estimators = 40,
                          colsample_bytree = 0.6)

lgbm_tuned = lgbm_tuned.fit(X_train,y_train)

In [12]:
y_pred=lgbm_tuned.predict(X_test)

In [14]:
np.sqrt(mean_squared_error(y_test,y_pred))

370.0492876350467